In [1]:
import pandas as pd
import numpy as np
import requests as re
import json
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
df = pd.read_csv("regulons+identifiers.csv")

In [12]:
base_url = "https://www.wikidata.org/w/rest.php/wikibase/v0"
request_item_by_id = "/entities/items/"

In [13]:
dumpy = re.get(base_url + request_item_by_id + "Q6776585").json()

In [14]:
label = "ExuR"
data = re.get(f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={label}&language=en&format=json").json()

In [24]:
data["search"][0]

{'id': 'Q23152600',
 'title': 'Q23152600',
 'pageid': 25163236,
 'display': {'label': {'value': 'DNA-binding transcriptional repressor ECUMN_3578',
   'language': 'en'},
  'description': {'value': 'microbial gene found in Escherichia coli UMN026',
   'language': 'en'}},
 'repository': 'wikidata',
 'url': '//www.wikidata.org/wiki/Q23152600',
 'concepturi': 'http://www.wikidata.org/entity/Q23152600',
 'label': 'DNA-binding transcriptional repressor ECUMN_3578',
 'description': 'microbial gene found in Escherichia coli UMN026',
 'match': {'type': 'alias', 'language': 'en', 'text': 'exuR'},
 'aliases': ['exuR']}

In [ ]:
# nach der wikidata id suchen anhand der namen der regulons und der otherdbsgeneids

In [6]:
def get_regulon_id_by_label_geneid(label, otherDbsGeneIds):
    otherDbsGeneIds = pd.Series(otherDbsGeneIds.split("]")).str[1:][:-1].str.split(":").str[1]
    responses = re.get(f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={label.lower()}&language=en&format=json").json()
    for response in responses["search"]:
        for otherDbsGeneId in otherDbsGeneIds:
            if response["label"].split(" ")[-1].lower() == otherDbsGeneId.lower():
                return response["id"].lower()

In [7]:
df["wikidata_ids"] = np.nan

In [8]:
for i in range(0, len(df.index)):
    df.loc[i, "wikidata_ids"] = get_regulon_id_by_label_geneid(df.iloc[i]["regulon_name"], df.iloc[i]["otherDbsGeneIds"])

In [9]:
len(df["wikidata_ids"].unique())

151

In [10]:
len(df.index)

265

In [11]:
not_in_wikidata = df[df["wikidata_ids"] != df["wikidata_ids"]].reset_index(drop = True)

In [12]:
not_in_wikidata

,regulon_name,regulated_genes,regulated_operon,regulated_tu,total_of_regulatory_interaction,total_of_binding_sites,geneId,leftEndPos,rightEndPos,strand,geneSynonyms,otherDbsGeneIds,productId,productName,productSynonyms,otherDbsProductsIds,wikidata_ids
0,asnc,4,2,4,8,8,RDBECOLIGNC00089,3926545.0,3927003.0,reverse,EG10093,[STRING:511145.b3743][ASAP:ABE-0012237][ECHOBA...,RDBECOLIPDC03932,DNA-binding transcriptional dual regulator AsnC,AsnC,[ALPHAFOLD:P0ACI6][PFAM:PF01037][INTERPRO:IPR0...,None
1,agar,11,3,3,11,11,RDBECOLIGNC04427,3277856.0,3278665.0,reverse,"yhaW,G7630",[STRING:511145.b3131][ASAP:ABE-0010291][ECHOBA...,RDBECOLIPDC03014,DNA-binding transcriptional repressor AgaR,"YhaW,AgaR",[ALPHAFOLD:P0ACK2][INTERPRO:IPR014036][PFAM:PF...,None
2,atoc,4,1,1,2,2,RDBECOLIGNC01609,2321866.0,2323251.0,forward,EG11668,[STRING:511145.b2220][ASAP:ABE-0007343][ECHOBA...,RDBECOLIPDC00142,DNA-binding transcriptional activator/ornithin...,"AtoC,AtoC response regulator,AtoC transcriptio...",[ALPHAFOLD:Q06065][REFSEQ:NP_416724][PFAM:PF02...,None
3,narl,125,34,44,127,97,RDBECOLIGNC00635,1275179.0,1275829.0,reverse,"frdR,narR,EG10643",[STRING:511145.b1221][ASAP:ABE-0004097][ECHOBA...,RDBECOLIPDC03848,DNA-binding transcriptional dual regulator NarL,"NarR,NarL,nitrate/nitrite response regulator N...",[ALPHAFOLD:P0AF28][REFSEQ:NP_415739][INTERPRO:...,None
4,fliz,22,9,17,18,10,RDBECOLIGNC01324,2000473.0,2001024.0,reverse,"yedH,EG11356",[STRING:511145.b1921][ASAP:ABE-0006394][ECHOBA...,RDBECOLIPDC00981,DNA-binding transcriptional regulator FliZ,"YedH,FliZ",[ALPHAFOLD:P52627][PFAM:PF02899][INTERPRO:IPR0...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,fimr2,5,0,0,5,5,RDBECOLIGNC04739,4543674.0,4543708.0,forward,G0-17115,[ECOCYC:G0-17115][REFSEQ:b4838],RDBECOLIPDC04657,small regulatory RNA FimR2,"FimR2,sRNA_35",[ECOCYC:RNA0-432],None
111,glnz,2,0,0,2,2,RDBECOLIGNC04740,4056428.0,4056621.0,reverse,G0-17117,[ECOCYC:G0-17117][REFSEQ:b4836],RDBECOLIPDC04658,small regulatory RNA GlnZ,"glnA 3<sup>A</sup> UTR,GlnZ",[ECOCYC:RNA0-434],None
112,asflhd,1,0,0,1,1,RDBECOLIGNC04741,1978176.0,1978395.0,forward,G0-17119,[ECOCYC:G0-17119][REFSEQ:b4839],RDBECOLIPDC04659,small antisense RNA AsflhD,AsflhD,[ECOCYC:RNA0-435],None
113,stfz,1,0,0,1,1,RDBECOLIGNC04742,104948.0,105499.0,reverse,G0-17120,[ECOCYC:G0-17120][REFSEQ:b4840],RDBECOLIPDC04660,small antisense RNA StfZ,StfZ,[ECOCYC:RNA0-436],None


In [145]:
def transform_json(jsonn):
    jsonn_str = str(jsonn)
    return jsonn_str.replace("'", "\"")

In [154]:
import requests
# Set up your OAuth credentials
self.username = ""
self.password = ""
wikidata_api_url = "https://www.wikidata.org/w/api.php"

# Step 1: Get an OAuth token
session = requests.Session()
login_token_request = session.get(
    wikidata_api_url,
    params={
        "action": "query",
        "meta": "tokens",
        "type": "login",
        "format": "json",
    },
)
login_token = login_token_request.json()["query"]["tokens"]["logintoken"]

# Step 2: Log in to get a session token
login_request = session.post(
    wikidata_api_url,
    data={
        "action": "login",
        "lgname": username,
        "lgpassword": password,
        "lgtoken": login_token,
        "format": "json",
    },
)
assert login_request.json()["login"]["result"] == "Success"

# Step 3: Get an edit token
edit_token_request = session.get(
    wikidata_api_url,
    params={
        "action": "query",
        "meta": "tokens",
        "format": "json",
    },
)
edit_token = edit_token_request.json()["query"]["tokens"]["csrftoken"]

# Step 4: Create a new item
new_item_data = {
    "labels": {"en": {"language" : "en", "value" : "minz pracus"}},
    "descriptions": {"en": {"language" : "en", "value" : "gambling king"}},
}



create_item_request = session.post(
    wikidata_api_url,
    data={
    "action": "wbeditentity",
    "format": "json",
    "new": "item",
    "token": edit_token,
    "data": transform_json(new_item_data),
    "formatversion": "2"
},
)
created_item_id = create_item_request.json().get("entity", {}).get("id")

# Display the created item ID
print("Created item ID:", created_item_id)

Created item ID: Q124267477


In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
import requests
import json

class WikidataClient:
    
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.wikidata_api_url = "https://www.wikidata.org/w/api.php"
        self.session = requests.Session()
        self.login_token = self.authenticate()
        self.edit_token = self.get_edit_token()
    
    def transform_json(self, jsonn):
        jsonn_str = str(jsonn)
        return jsonn_str.replace("'", "\"")

    def authenticate(self):
        login_token_response = self.session.get(
            self.wikidata_api_url,
            params={
                "action": "query",
                "meta": "tokens",
                "type": "login",
                "format": "json",
            },
        )
        login_token = login_token_response.json()["query"]["tokens"]["logintoken"]
        return login_token
        
    def get_session_token(self):
        login_response = self.session.post(
            self.wikidata_api_url,
            data={
                "action": "login",
                "lgname": self.username,
                "lgpassword": self.password,
                "lgtoken": self.login_token,
                "format": "json",
            },
        )
        assert login_response.json()["login"]["result"] == "Success"
    
    def get_edit_token(self):
        edit_token_response = self.session.get(
            self.wikidata_api_url,
            params={
                "action": "query",
                "meta": "tokens",
                "format": "json",
            },
        )
        edit_token = edit_token_response.json()["query"]["tokens"]["csrftoken"]
        return edit_token

    def create_new_item(self, language, label, description):
        new_item_data = {
            "labels": {language: {"language" : language, "value" : label}},
            "descriptions": {language: {"language" : language, "value" : description}},
        }
        create_item_response = self.session.post(
            self.wikidata_api_url,
            data={
                "action": "wbeditentity",
                "format": "json",
                "new": "item",
                "token": self.edit_token,
                "data": self.transform_json(new_item_data),
                "formatversion": "2"
            },
        )
        created_item_id = create_item_response.json().get("entity", {}).get("id")
        return created_item_id
    
    def get_entity_by_id(self, ID, language):
        get_entity_response = self.session.get(
            self.wikidata_api_url,
            params={
                "action": "wbgetentities",
                "format": "json",
                "ids": ID,
                "languages": language,
                "formatversion": "2"
            },
        )
        return get_entity_response.json()
    

In [4]:
client = WikidataClient()

In [72]:
create_item_request

<Response [200]>

In [16]:
login_token

'd7fa3286ce9067c5320eb9c187627fa265a2b4de+\\'

In [17]:
edit_token

'6a0c01deaf429734df710b4533ebc64e65a2b4df+\\'

In [94]:
create_item_request.json()

{'entity': {'type': 'item',
  'id': 'Q124262825',
  'labels': {'de': {'language': 'de', 'value': 'de-value'},
   'en': {'language': 'en', 'value': 'en-value'}},
  'descriptions': {},
  'aliases': {},
  'claims': {},
  'sitelinks': {},
  'lastrevid': 2052878251},
 'success': 1}

In [28]:
created_item_id